In [28]:
import argparse
import torch

In [29]:
from PIL import Image
import os
import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer
from tqdm import tqdm

In [30]:
from transformers import logging
from sampling.Myspeculative_sampling import speculative_sampling
from sampling.Myautoregressive_sampling import autoregressive_sampling

In [31]:
logging.set_verbosity_error()

In [32]:
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig,AutoProcessor

In [33]:
def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

In [34]:
def disable_torch_init():
    """
    Disable the redundant torch default initialization to accelerate model creation.
    """
    import torch
    setattr(torch.nn.Linear, "reset_parameters", lambda self: None)
    setattr(torch.nn.LayerNorm, "reset_parameters", lambda self: None)

In [36]:
def get_model_name_from_path(model_path):
    model_path = model_path.strip("/")
    model_paths = model_path.split("/")
    if model_paths[-1].startswith('checkpoint-'):
        return model_paths[-2] + "_" + model_paths[-1]
    else:
        return model_paths[-1]

In [37]:
def main(args):
    disable_torch_init()

    gamma=args.gamma

    top_k = 1
    top_p = 0.9
    temperature=args.temperature

    random_seed=args.seed

    device_map="cuda:2"
    kwargs = {"device_map": device_map, **kwargs}

    print('Loading  draft  model...')
  
    draft_model = Qwen2VLForConditionalGeneration.from_pretrained(args.draft_model_path,**kwargs)

    print('Loading  target  model...')

    target_model = Qwen2VLForConditionalGeneration.from_pretrained(args.target_model_path,**kwargs)
    tokenizer = AutoTokenizer.from_pretrained(args.target_model_path, use_fast=True)
    processor = AutoProcessor.from_pretrained(args.target_model_path)

    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

    max_len=args.max_new_tokens
    

In [40]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='test')
    parser.add_argument('--draft_model_path', type=str, default="/data/share/model_weight/qwen/Qwen2.5-0.5B")
    parser.add_argument('--target_model_path', type=str, default="/data/share/model_weight/qwen/Qwen2.5-7B")
    parser.add_argument("--temperature", type=float, default=1)
    parser.add_argument("--max_new_tokens", type=int, default=30)
    parser.add_argument('--seed', '-s', type=int, default=66, help='set a random seed, which can makes the result reproducible')
    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--draft_model_path DRAFT_MODEL_PATH]
                             [--target_model_path TARGET_MODEL_PATH]
                             [--temperature TEMPERATURE]
                             [--max_new_tokens MAX_NEW_TOKENS] [--seed SEED]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/bks/.local/share/jupyter/runtime/kernel-v38e9fbe4f33c9ad54c9e34a8a31d66f5a581a7fb1.json


SystemExit: 2